In [1]:
import networkx as nx

def load_graph(edges_file):
    """Loads a graph from an edge list file."""
    G = nx.Graph()  # Or nx.DiGraph() for directed graphs
    with open(edges_file, 'r') as f:
        for line in f:
            u, v = map(int, line.strip().split()) # Assuming nodes are integers separated by space
            G.add_edge(u, v)
    return G



# Example Usage (replace with your actual file and nodes):
edges_file = "/kaggle/input/facebook-combined/facebook_combined.txt" # Replace with the name of your edge list file
graph = load_graph(edges_file)


In [2]:
def find_shortest_path(graph, start_node, end_node):
    """Finds the shortest path between two nodes in a graph."""
    try:
        path = nx.shortest_path(graph, source=start_node, target=end_node)
        return path
    except nx.NetworkXNoPath:
        return None  # No path exists
    
start_node = 0
end_node = 352

shortest_path = find_shortest_path(graph, start_node, end_node)

if shortest_path:
    print(f"Shortest path from {start_node} to {end_node}: {shortest_path}")
else:
    print(f"No path found between {start_node} and {end_node}.")

Shortest path from 0 to 352: [0, 34, 348, 352]


In [3]:
def find_friends_within_distance(graph, center_node, max_distance):
    """Finds nodes within a specified distance from a central node."""
    friends = []
    for node, dist in nx.single_source_shortest_path_length(graph, center_node).items():
        if dist <= max_distance and node != center_node:  # Exclude the center node itself
            friends.append(node)
    return friends

center_node = 414  # The node you want to find friends for
max_distance = 2 # Find nodes within a distance of 2 hops

friends = find_friends_within_distance(graph, center_node, max_distance)
print(f"Number of friend: {len(friends)}")

print(f"Friends of {center_node} within distance {max_distance}: {friends}")

Number of friend: 1376
Friends of 414 within distance 2: [34, 107, 173, 348, 363, 370, 373, 374, 376, 378, 391, 394, 395, 400, 412, 422, 423, 427, 428, 431, 434, 436, 438, 461, 465, 475, 480, 483, 492, 496, 500, 506, 513, 514, 515, 524, 542, 544, 553, 556, 558, 559, 561, 563, 566, 567, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 0, 58, 171, 353, 366, 389, 420, 484, 517, 526, 538, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 9

In [4]:
def find_most_popular_nodes(graph, n):
    """Finds the top N nodes with the highest degree (most connections)."""
    degrees = dict(graph.degree())  # Calculate degree of each node
    sorted_degrees = sorted(degrees.items(), key=lambda item: item[1], reverse=True) # Sort by degree descending

    top_n_nodes = [node for node, degree in sorted_degrees[:n]] # Take the top N nodes

    return top_n_nodes

def find_least_popular_nodes(graph, n):
    """Finds the top N nodes with the lowest degree (fewest connections)."""
    degrees = dict(graph.degree())  # Calculate degree of each node
    sorted_degrees = sorted(degrees.items(), key=lambda item: item[1]) # Sort by degree ascending

    top_n_nodes = [node for node, degree in sorted_degrees[:n]] # Take the top N nodes

    return top_n_nodes

n_top_nodes = 5
most_popular_nodes = find_most_popular_nodes(graph,n_top_nodes)
least_popular_nodes = find_least_popular_nodes(graph,n_top_nodes)

print(f"Node(s) with the most friends: {most_popular_nodes}")
print(f"Node(s) with the least friends: {least_popular_nodes}")

Node(s) with the most friends: [107, 1684, 1912, 3437, 0]
Node(s) with the least friends: [11, 12, 15, 18, 37]


In [5]:
def extract_features(G, u, v):
    return {
        'common_neighbors': len(list(nx.common_neighbors(G, u, v))),
        'jaccard': list(nx.jaccard_coefficient(G, [(u, v)]))[0][2],
        'adamic_adar': list(nx.adamic_adar_index(G, [(u, v)]))[0][2],
        'preferential_attachment': list(nx.preferential_attachment(G, [(u, v)]))[0][2],
        'shortest_path': nx.shortest_path_length(G, u, v) if nx.has_path(G, u, v) else 99
    }



In [6]:
import networkx as nx
import random
import math
import csv
import datetime
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import multiprocessing as mp
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
import numpy as np
from sklearn import linear_model
from sklearn.metrics import log_loss
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.linear_model import LogisticRegression
def CommonNeighbors(u, v, g):
    u_neighbors = set(g.neighbors(u))
    v_neighbors = set(g.neighbors(v))
    return len(u_neighbors.intersection(v_neighbors))
def common_neighbors(g, edges):
    result = []
    for edge in edges:
        node_one, node_two = edge[0], edge[1]
        num_common_neighbors = 0
        try:
            neighbors_one, neighbors_two = g.neighbors(node_one), g.neighbors(node_two)
            for neighbor in neighbors_one:
                if neighbor in neighbors_two:
                    num_common_neighbors += 1
            result.append((node_one, node_two, num_common_neighbors))
        except:
            pass
    return result
def AdamicAdar(u, v, g):
    u_neighbors = set(g.neighbors(u))
    v_neighbors = set(g.neighbors(v))
    aa = 0
    for i in u_neighbors.intersection(v_neighbors):
        aa += 1 / math.log(len(g.neighbors(i)))
    return aa

def ResourceAllocation(u, v, g):
    u_neighbors = set(g.neighbors(u))
    v_neighbors = set(g.neighbors(v))
    ra = 0
    for i in u_neighbors.intersection(v_neighbors):
        ra += 1 / float(len(g.neighbors(i)))
    return ra
def JaccardCoefficent(u, v, g):
    u_neighbors = set(g.neighbors(u))
    v_neighbors = set(g.neighbors(v))
    return len(u_neighbors.intersection(v_neighbors)) / float(len(u_neighbors.union(v_neighbors)))

def PreferentialAttachment(u, v, g):
    return len(g.neighbors(u))*len(g.neighbors(v))

def AllFeatures(u,v,g1, g2):
    '''
    the change of features in two consecutive sub graphs
    '''
    try:
        cn = CommonNeighbors(u, v, g2)
        aa = AdamicAdar(u, v, g2)
        ra = ResourceAllocation(u, v, g2)
        jc = JaccardCoefficent(u, v, g2)
        pa = PreferentialAttachment(u, v, g2)

        delta_cn = cn - CommonNeighbors(u, v, g1)
        delta_aa = aa - AdamicAdar(u, v, g1)
        delta_ra = ra - ResourceAllocation(u, v, g1)
        delta_jc = jc - JaccardCoefficent(u, v, g1)
        delta_pa = pa - PreferentialAttachment(u, v, g1)
        return {"cn":cn, "aa": aa, "ra":ra, "jc":jc, "pa":pa,
            "delta_cn": delta_cn, "delta_aa": delta_aa, "delta_ra": delta_ra,
             "delta_jc": delta_jc, "delta_pa": delta_pa}
    except:
        pass
feature_set = [common_neighbors,
                   nx.resource_allocation_index,
                   nx.jaccard_coefficient,
                   nx.adamic_adar_index,
                   nx.preferential_attachment
                   ]
def produce_fake_edge(g, neg_g,num_test_edges):
    i = 0
    while i < num_test_edges:
        edge = random.sample(list(g.nodes()), 2)
        try:
            shortest_path = nx.shortest_path_length(g,source=edge[0],target=edge[1])
            if shortest_path >= 2:
                neg_g.add_edge(edge[0],edge[1], positive="False")
                i += 1
        except:
            pass

def create_graph_from_file(filename):
    print("----------------build graph--------------------")
    f = open(filename, "rb")
    g = nx.read_edgelist(f)
    return g

def sample_extraction(g, pos_num, neg_num, neg_mode, neg_distance=2, delete=1):
    """

    :param g:  the graph
    :param pos_num: the number of positive samples
    :param neg_num: the number of negative samples
    :param neg_distance: the distance between two nodes in negative samples
    :param delete: if delete ==0, don't delete positive edges from graph
    :return: pos_sample is a list of positive edges, neg_sample is a list of negative edges
    """

    print("----------------extract positive samples--------------------")
    # randomly select pos_num as test edges
    pos_sample = random.sample(list(g.edges()), pos_num)
    sample_g = nx.Graph()
    sample_g.add_edges_from(pos_sample, positive="True")
    nx.write_edgelist(sample_g, "sample_positive_" +str(pos_num)+ ".txt", data=['positive'])

    # adding non-existing edges
    print("----------------extract negative samples--------------------")
    i = 0
    neg_g = nx.Graph()
    produce_fake_edge(g,neg_g,neg_num)
    nx.write_edgelist(neg_g, "sample_negative_" +str(neg_num)+ ".txt", data=["positive"])
    neg_sample = neg_g.edges()
    neg_g.add_edges_from(pos_sample,positive="True")
    nx.write_edgelist(neg_g, "sample_combine_" +str(pos_num + neg_num)+ ".txt", data=["positive"])

    # remove the positive sample edges, the rest is the training set
    if delete == 0:
        return pos_sample, neg_sample
    else:
        g.remove_edges_from(pos_sample)
        nx.write_edgelist(g, "training.txt", data=False)

        return pos_sample, neg_sample

def feature_extraction(g, pos_sample, neg_sample, feature_name, model="single", combine_num=5):

    data = []
    if model == "single":
        print ("-----extract feature:", feature_name.__name__, "----------")
        preds = feature_name(g, pos_sample)
        feature = [feature_name.__name__] + [i[2] for i in preds]
        label = ["label"] + ["Pos" for i in range(len(feature))]
        preds = feature_name(g, neg_sample)
        feature1 = [i[2] for i in preds]
        feature = feature + feature1
        label = label + ["Neg" for i in range(len(feature1))]
        data = [feature, label]
        data = transpose(data)
        print("----------write the feature to file---------------")
        write_data_to_file(data, "features_" + model + "_" + feature_name.__name__ + ".csv")
    else:
        label = ["label"] + ["1" for i in range(len(pos_sample))] + ["0" for i in range(len(neg_sample))]
        for j in feature_name:
            print ("-----extract feature:", j.__name__, "----------")
            preds = j(g, pos_sample)

            feature = [j.__name__] + [i[2] for i in preds]
            preds = j(g, neg_sample)
            feature = feature + [i[2] for i in preds]
            data.append(feature)

        data.append(label)
        data = transpose(data)
        print("----------write the features to file---------------")
        write_data_to_file(data, "features_" + model + "_" + str(combine_num) + ".csv")
    return data
def write_data_to_file(data, filename):
    csvfile = open(filename, "w")
    writer = csv.writer(csvfile)
    for i in data:
        writer.writerow(i)
    csvfile.close()


def transpose(data):
    return [list(i) for i in zip(*data)]

def main(filename="/kaggle/input/facebook-combined/facebook_combined.txt", pos_num=0.1, neg_num=0.1, model="combined", combine_num=1,
         feature_name=common_neighbors, neg_mode="hard"):
    if combine_num==2:
        pos_num=0.008;
        neg_num=0.008;
    g = create_graph_from_file(filename)
    num_edges = g.number_of_edges()
    pos_num = int(num_edges * pos_num)
    neg_num = int(num_edges * neg_num)
    pos_sample, neg_sample = sample_extraction(g, pos_num, neg_num,neg_mode)
    train_data = feature_extraction(g, pos_sample, neg_sample, feature_name, model, combine_num)


#SOLUTION:
fn="/kaggle/input/facebook-combined/facebook_combined.txt";
cn=2;
#Run this line to genrate feature Set
main(filename=fn,model="combined",combine_num=cn, feature_name=feature_set, neg_mode="easy")

r=np.loadtxt(open("features_combined_"+str(cn)+".csv", "rb"), delimiter=",", skiprows=1);
l,b=r.shape;
np.random.shuffle(r);
train_l=int(0.75*l)
X_train=r[0:train_l,0:b-1]
Y_train=r[0:train_l,b-1]
X_test=r[train_l:l,0:b-1]
Y_test=r[train_l:l,b-1]
X_train = normalize(X_train, axis=0, norm='max')
X_test = normalize(X_test, axis=0, norm='max')
scaler = StandardScaler()  
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  
def mySvm(training, training_labels, testing, testing_labels):
    #Support Vector Machine
    start = datetime.datetime.now()
    clf = svm.SVC()
    clf.fit(training, training_labels)
    print ("+++++++++ Finishing training the SVM classifier ++++++++++++")
    result = clf.predict(testing)

    print ("SVM accuracy:", accuracy_score(testing_labels, result))
    #keep the time
    finish = datetime.datetime.now()
    print ((finish-start).seconds)
mySvm(X_train,Y_train,X_test,Y_test)

def logistic(training, training_labels, testing, testing_labels):
    clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='ovr').fit(training, training_labels)
    start = datetime.datetime.now()
    clf.fit(training, training_labels)
    result=clf.predict(testing)
    print ("+++++++++ Finishing training the Linear classifier ++++++++++++")
    print ("Linear accuracy:", accuracy_score(testing_labels, result))
    #keep the time
    finish = datetime.datetime.now()
    print ((finish-start).seconds)

logistic(X_train,Y_train,X_test,Y_test)

def ANN(training, training_labels, testing, testing_labels):
    clf = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(15,9), random_state=1)
    start = datetime.datetime.now()
    clf.fit(training, training_labels)
    print ("+++++++++ Finishing training the ANN classifier ++++++++++++")
    result = clf.predict(testing)

    print ("ANN accuracy:", accuracy_score(testing_labels, result))
    #keep the time
    finish = datetime.datetime.now()
    print ((finish-start).seconds)
ANN(X_train,Y_train,X_test,Y_test)

----------------build graph--------------------
----------------extract positive samples--------------------
----------------extract negative samples--------------------
-----extract feature: common_neighbors ----------
-----extract feature: resource_allocation_index ----------
-----extract feature: jaccard_coefficient ----------
-----extract feature: adamic_adar_index ----------
-----extract feature: preferential_attachment ----------
----------write the features to file---------------
+++++++++ Finishing training the SVM classifier ++++++++++++
SVM accuracy: 0.6521739130434783
0
+++++++++ Finishing training the Linear classifier ++++++++++++
Linear accuracy: 0.6616257088846881
0
+++++++++ Finishing training the ANN classifier ++++++++++++
ANN accuracy: 0.6786389413988658
0


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [7]:
import networkx as nx
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# 1️⃣ Tạo đồ thị từ file txt
def create_graph_from_file(path):
    G = nx.read_edgelist(path)
    print(f"Đồ thị có {G.number_of_nodes()} đỉnh và {G.number_of_edges()} cạnh.")
    return G

# 2️⃣ Trích xuất đặc trưng đồ thị giữa cặp đỉnh
def extract_features(G, u, v):
    return [
        len(list(nx.common_neighbors(G, u, v))),
        list(nx.jaccard_coefficient(G, [(u, v)]))[0][2],
        list(nx.adamic_adar_index(G, [(u, v)]))[0][2],
        list(nx.preferential_attachment(G, [(u, v)]))[0][2]
    ]

# 3️⃣ Tạo tập train từ đồ thị
def generate_training_data(G, num_samples=10000):
    nodes = list(G.nodes())
    edges = list(G.edges())
    X = []
    y = []

    # Positive samples: các cặp có cạnh
    positive_edges = random.sample(edges, min(len(edges), num_samples // 2))
    for u, v in positive_edges:
        try:
            X.append(extract_features(G, u, v))
            y.append(1)
        except:
            continue

    # Negative samples: các cặp không có cạnh
    count = 0
    while count < len(positive_edges):
        u, v = random.sample(nodes, 2)
        if not G.has_edge(u, v):
            try:
                X.append(extract_features(G, u, v))
                y.append(0)
                count += 1
            except:
                continue

    print(f"Tạo xong {len(X)} mẫu training.")
    return np.array(X), np.array(y)

# 4️⃣ Huấn luyện mô hình ML
def train_model(X, y):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X, y)
    return model

# 5️⃣ Đề xuất bạn bè cho 1 node
def suggest_friends(G, model, target_node, top_k=5):
    if target_node not in G.nodes():
        print(f"Node {target_node} không tồn tại trong đồ thị.")
        return []

    suggestions = []
    for node in G.nodes():
        if node != target_node and not G.has_edge(target_node, node):
            try:
                features = extract_features(G, target_node, node)
                prob = model.predict_proba([features])[0][1]  # Xác suất là bạn
                suggestions.append((node, prob))
            except:
                continue

    # Sắp xếp theo xác suất giảm dần
    suggestions.sort(key=lambda x: x[1], reverse=True)
    return suggestions[:top_k]

# 6️⃣ TOÀN BỘ QUY TRÌNH
path_to_txt = "/kaggle/input/facebook-combined/facebook_combined.txt"  # thay đổi đường dẫn nếu cần
G = create_graph_from_file(path_to_txt)

X, y = generate_training_data(G, num_samples=2000)
model = train_model(X, y)


Đồ thị có 4039 đỉnh và 88234 cạnh.
Tạo xong 2000 mẫu training.


In [8]:
extract_features(G, "0", "107")

[2, 0.0014388489208633094, 0.7259450575305171, 362615]

In [9]:
def suggest_friends(G, model, target_node, top_k=10):
    neighbors = set(G.neighbors(target_node))  # bạn bè trực tiếp
    candidates = set()

    for friend in neighbors:
        second_neighbors = set(G.neighbors(friend))  # bạn của bạn
        candidates.update(second_neighbors)

    candidates -= neighbors  # loại bỏ bạn đã kết nối rồi
    candidates.discard(target_node)  # loại bỏ chính mình

    suggestions = []
    for v in candidates:
        X = extract_features(G, target_node, v)
        prob = model.predict_proba([X])[0][1]
        suggestions.append((v, prob))

    suggestions.sort(key=lambda x: x[1], reverse=True)
    return suggestions[:top_k], neighbors

target_node = "107"
suggested, neighbors = suggest_friends(G, model, target_node, top_k=10)

# In kết quả
print(f"\nTop 10 gợi ý bạn bè cho node {target_node}:")
for friend, prob in suggested:
    # Kiểm tra xem node được gợi ý có là bạn của bạn nào không
    common_friends = [n for n in neighbors if G.has_edge(friend, n)]
    
    print(f"- Node {friend} với xác suất {prob:.4f}", end=' ')
    if common_friends:
        print(f"=> Là bạn của {len(common_friends)} bạn của node {target_node}: {common_friends}")
    else:
        print("=> ❌ Không là bạn của bạn nào cả")

print("\nBạn bè trực tiếp của node", target_node, ":", neighbors)



Top 10 gợi ý bạn bè cho node 107:
- Node 378 với xác suất 1.0000 => Là bạn của 16 bạn của node 107: ['563', '517', '428', '363', '483', '484', '376', '580', '641', '366', '348', '414', '637', '475', '651', '566']
- Node 465 với xác suất 1.0000 => Là bạn của 14 bạn của node 107: ['538', '420', '517', '428', '601', '483', '484', '526', '376', '580', '348', '414', '475', '566']
- Node 461 với xác suất 1.0000 => Là bạn của 9 bạn của node 107: ['517', '606', '428', '363', '483', '376', '348', '414', '475']
- Node 520 với xác suất 1.0000 => Là bạn của 9 bạn của node 107: ['353', '563', '517', '428', '483', '484', '376', '366', '348']
- Node 500 với xác suất 1.0000 => Là bạn của 17 bạn của node 107: ['353', '538', '563', '517', '428', '601', '363', '483', '484', '376', '580', '348', '414', '637', '475', '651', '566']
- Node 373 với xác suất 1.0000 => Là bạn của 17 bạn của node 107: ['353', '563', '517', '606', '428', '363', '483', '376', '580', '641', '366', '348', '414', '637', '475', '651'